<a href="https://colab.research.google.com/github/fabarroso/MVP-ML/blob/main/mvpml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MVP -  Machine Learning
Nome: Fabio de Andrade Barroso

Matricula:4052025000158

Dataset original: https://basedosdados.org/dataset/dbd717cb-7da8-4efd-9162-951a71694541?table=a2e9f998-e2c2-49b7-858a-ae1daef46dc0

**Segurança no Estado de São Paulo - Dados estatísticos da Secretaria de Segurança Pública do Estado de São Paulo.**

O dataset contém informações sobre ocorrências policiais no estado de São Paulo, com diversas variáveis relacionadas a diferentes tipos de crimes (homicídios, furtos, roubos, estupros, etc.) por município, mês e ano.

**Organização:**
Governo de São Paulo

**Cobertura temporal:**
2002 - 2021

**1 - Objetivo**

Analisar o comportamento e a ocorrência de crimes ao longo do tempo, utilizando técnicas de Análise Exploratória de Dados (EDA), pré-processamento de dados e aprendizado de máquina para identificar padrões, prever tendências e possíveis fatores que impactam a criminalidade.

**1.1 - Escopo**

**Exploração dos dados:** Identificar e compreender a distribuição e tendências dos diferentes tipos de crimes (homicídios, roubos, furtos, etc.) ao longo do tempo.

**Pré-processamento:** Lidar com valores ausentes, formatação de dados, normalização e transformação de variáveis para melhor adequação ao modelo.

**Machine Learning:** Aplicar modelos preditivos para classificar ou prever tipos de crimes ou áreas de maior risco com base em variáveis temporais e geográficas.

**1.2 - Contexto do Problema**

A segurança pública é um tema central em muitas discussões políticas e sociais. A análise dos dados de criminalidade pode auxiliar no planejamento de políticas públicas mais eficazes e estratégias de prevenção. Com o uso de machine learning, é possível prever áreas de maior risco e padrões sazonais de crimes, ajudando as autoridades a direcionar recursos de forma mais eficiente.

**2 - Ambiente**

As bibliotecas foram escolhidas para garantir um fluxo eficiente de análise de dados e modelagem. *Pandas* e *NumPy* são usadas para manipulação e cálculo dos dados. *Matplotlib* e *Seaborn* são utilizadas para visualização e exploração gráfica. *Scikit-learn*  facilita o pré-processamento, criação de modelos, validação cruzada e avaliação de desempenho. Essas ferramentas asseguram uma análise robusta e reprodutível do dataset.

Ao carregar o dataset e realizar experimentos com o modelo, a definição da *seed*   assegura que a divisão entre treino e teste e outras etapas aleatórias, como a inicialização do modelo, sejam consistentes. Isso é crucial para garantir que as avaliações de desempenho do modelo sejam justas e reproduzíveis em diferentes execuções.

Foi utilizado pd.set_option do Pandas, evitando assim que colunas ou textos ficassem truncados.

In [69]:
## Carga de bibliotecas utilizadas

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import random
import sys
#from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Definir o SEED para reprodutibilidade (controle de aleatoriedade)
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Ajustando as configurações para exibir todas as colunas e linhas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)

# Exibição de resultados
print("Python:", sys.version.split()[0])
print("Seed global:", SEED)

Python: 3.12.11
Seed global: 42


**3 - Dados: carga, entendimento e qualidade**

O dataset foi carregado diretamente de uma URL no GitHub, utilizando pd.read_csv, com a configuração de delimitador para vírgula (,) e codificação adequada para caracteres especiais (ISO-8859-1).

In [70]:
##Carregamento dos Dados
# URL GitHub
url = 'https://raw.githubusercontent.com/fabarroso/dados_sp_gov_ssp/main/sp_gov_ssp.csv'

# Carregamento do dataset
df = pd.read_csv(url, delimiter=',', encoding='ISO-8859-1')

# Substituindo valores NaN por um placeholder ("N/A") para evitar quebras no template
df.fillna('N/A', inplace=True)

# Exibe as primeiras 10 linhas
print(df.head(10).to_string())

    ano  mes  id_municipio           regiao_ssp homicidio_doloso numero_de_vitimas_em_homicidio_doloso homicidio_doloso_por_acidente_de_transito numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito homicidio_culposo_por_acidente_de_transito homicidio_culposo_outros tentativa_de_homicidio lesao_corporal_seguida_de_morte lesao_corporal_dolosa lesao_corporal_culposa_por_acidente_de_transito lesao_corporal_culposa_outras latrocinio numero_de_vitimas_em_latrocinio total_de_estupro estupro estupro_de_vulneravel total_de_roubo_outros roubo_outros roubo_de_veiculo roubo_a_banco roubo_de_carga furto_outros furto_de_veiculo
0  2002    1       3500105  Presidente Prudente             0.00                                  0.00                                      0.00                                                           0.00                                       0.00                     0.00                   0.00                            0.00                 24.00                

os ultimos 4 meses do ultimo ano estão com as linhas vazias


In [71]:
# Primeiramente, identificamos o último ano no dataset
ultimo_ano = df['ano'].max()

# Identificar os últimos 4 meses desse último ano
meses_para_excluir = [9, 10, 11, 12]  # setembro, outubro, novembro, dezembro

# Criar um DataFrame com os dados que serão excluídos (últimos 4 meses do último ano)
df_eliminado = df[(df['ano'] == ultimo_ano) & (df['mes'].isin(meses_para_excluir))]

# Exibir as linhas eliminadas
print("Linhas Eliminadas:")
print(df_eliminado)

print("\n" + "-"*50 + "\n")

# Exibir a quantidade de linhas eliminadas
print("\nQuantidade de Linhas Eliminadas:", df_eliminado.shape[0])

# Filtrando o DataFrame para excluir os últimos 4 meses do último ano
df_filtrado = df[~((df['ano'] == ultimo_ano) & (df['mes'].isin(meses_para_excluir)))]

print("\n" + "-"*50 + "\n")

# Informações dos tipos de dados
df.info()

print("\n" + "-"*50 + "\n")

# Estatísticas descritivas
print (df.describe())
#print(df.describe(include='all'))

Linhas Eliminadas:
         ano  mes  id_municipio               regiao_ssp homicidio_doloso numero_de_vitimas_em_homicidio_doloso homicidio_doloso_por_acidente_de_transito numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito homicidio_culposo_por_acidente_de_transito homicidio_culposo_outros tentativa_de_homicidio lesao_corporal_seguida_de_morte lesao_corporal_dolosa lesao_corporal_culposa_por_acidente_de_transito lesao_corporal_culposa_outras latrocinio numero_de_vitimas_em_latrocinio total_de_estupro estupro estupro_de_vulneravel total_de_roubo_outros roubo_outros roubo_de_veiculo roubo_a_banco roubo_de_carga furto_outros furto_de_veiculo
123056  2021    9       3500402               Piracicaba              N/A                                   N/A                                       N/A                                                            N/A                                        N/A                      N/A                    N/A                             N/A 

In [78]:
# Criar um dataframe para mostrar a estrutura de forma tabular
column_structure = pd.DataFrame({
    'Nome da Coluna': df.columns,
    'Descrição': [column.replace('_', ' ').capitalize() for column in df.columns],
    'Tipo de Dado': [df[column].dtype for column in df.columns]
})

# Exibir a estrutura do dataset de forma tabular
#column_structure.style.set_table_attributes('class="table table-striped"').hide(axis="index")
column_structure

,Nome da Coluna,Descrição,Tipo de Dado
0,ano,Ano,int64
1,mes,Mes,int64
2,id_municipio,Id municipio,int64
3,regiao_ssp,Regiao ssp,object
4,homicidio_doloso,Homicidio doloso,object
5,numero_de_vitimas_em_homicidio_doloso,Numero de vitimas em homicidio doloso,object
6,homicidio_doloso_por_acidente_de_transito,Homicidio doloso por acidente de transito,object
7,numero_de_vitimas_em_homicidio_doloso_por_acidente_de_transito,Numero de vitimas em homicidio doloso por acidente de transito,object
8,homicidio_culposo_por_acidente_de_transito,Homicidio culposo por acidente de transito,object
9,homicidio_culposo_outros,Homicidio culposo outros,object


3.1 Análise exploratória (EDA)